In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.data_preprocess import * # data_split, DataLabeling, train_val_test_split
from models import *

In [2]:
WINDOW_SIZE = 24

In [3]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')
raw_data = DataLabeling(df, WINDOW_SIZE)
train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data)
Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)

In [6]:
model = CDT_1D_model(24, 5)

In [7]:
test_model = model.model

In [9]:
test_model.fit(Data.train,
               epochs=20,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path='without_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                          tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, min_lr=0.00001, factor=0.8)])

Epoch 1/20
11867/11867 [==============================] - 270s 23ms/step - loss: 0.7784 - accuracy: 0.5704 - val_loss: 99.0767 - val_accuracy: 0.0059
Epoch 2/20
11867/11867 [==============================] - 311s 26ms/step - loss: 0.7726 - accuracy: 0.5829 - val_loss: 285.6733 - val_accuracy: 0.4320
Epoch 3/20
11867/11867 [==============================] - 265s 22ms/step - loss: 0.7602 - accuracy: 0.5908 - val_loss: 82.7184 - val_accuracy: 0.5621
Epoch 4/20
11867/11867 [==============================] - 308s 26ms/step - loss: 0.7469 - accuracy: 0.5952 - val_loss: 46.7466 - val_accuracy: 0.5621
Epoch 5/20
11867/11867 [==============================] - 261s 22ms/step - loss: 0.7427 - accuracy: 0.5964 - val_loss: 28.7002 - val_accuracy: 0.0059
Epoch 6/20
11867/11867 [==============================] - 249s 21ms/step - loss: 0.7392 - accuracy: 0.5975 - val_loss: 3.1382 - val_accuracy: 0.5621
Epoch 7/20
11867/11867 [==============================] - 280s 24ms/step - loss: 0.7390 - accuracy: 

In [10]:
test_model.evaluate(Data.test)

7/7 [==============================] - 0s 5ms/step - loss: 2.3767 - accuracy: 0.7808


[2.3766791820526123, 0.7808219194412231]

In [22]:
tf.argmax(test_model.predict(Data.val), axis=1)

<tf.Tensor: shape=(169,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)>

In [23]:
val_df.Trend.value_counts()

2    104
0     87
1      1
Name: Trend, dtype: int64